# Importing the neccessary libraries, loading the required data, and training the model

In [1]:
# run the bot notebook to train the model and store it
%run bot.ipynb

# for proccessing the test data(chats from the user)
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# for loading data
import pickle
import json

# basic libraries to access and manipulate our arrays 
import numpy as np
import random

# loading the data
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

# loading the trained model
model.load('./model.tflearn')

Training Step: 11999  | total loss: 0.02099 | time: 0.033s
| Adam | epoch: 1000 | loss: 0.02099 - acc: 0.9948 -- iter: 88/90
Training Step: 12000  | total loss: 0.01930 | time: 0.036s
| Adam | epoch: 1000 | loss: 0.01930 - acc: 0.9953 -- iter: 90/90
--
INFO:tensorflow:c:\Users\himas\Downloads\Learning_projects\VS_code\Chat_bot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from c:\Users\himas\Downloads\Learning_projects\VS_code\Chat_bot\model.tflearn


# Functions for processing the test data

In [2]:
def get_words(sent):
    '''
    to break a sentence and get the processed words
    Parameters:
            sent: the sentence to be processed, a scalar string
    Returns:
            sentence_words: an array of lemmatized and lowered words tokenized from the sentence
    '''
    # tokenize the sentence
    token_w = nltk.word_tokenize(sent)
    # lemmatize and lower the words
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in token_w]
    return sentence_words

def vectorize_sent(sent, words, show_details=False):
    '''
    return bag of words vector, with 1 if word in the bag exists in the sentence, 0 otherwise
    Parameters:
            sent: the sentence to be processed, a scalar string
            words: an array of all the processed words present in the training document
            show_details: a boolean to indicate if the details involved in proccessing the data should be displayed for testing purposes
    Returns:
            a bag of words array that is given as input to the model (X_test)
    '''
    # get the words
    sentence_words = get_words(sent)
    # initialize the bag of words vector
    bag_of_words = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag_of_words[i] = 1
                if show_details:
                    print(f'\nFound in bag: {w}')

    return (np.array(bag_of_words))

# to check
print("\nInput sentence : \'How to apply for teaching position?\'")
p = vectorize_sent('How to apply for teaching position?', words, show_details=True)
print(f"\nbag of words: {p}")


Input sentence : 'How to apply for teaching position?'

Found in bag: apply

Found in bag: teaching

Found in bag: position

bag of words: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0]


# Functions to get response from the model 

In [3]:
# define the threshold: consider the response only if the model probability of that class is beyond the threshold
ERROR_THRESHOLD = 0.25

# create a data structure to hold user context
context = {}

def predict_class(sent):
    '''
    Obtain the likelihood of the given sentence belonging to one of the classes listed
    Parameters:
            sent: the sentence to be analyzed, a scalar string
    Returns:
            results: an array of tuples, representing the class and its probability of being the result for the sentence
    '''
    # generate probabilities using the model
    results = model.predict([vectorize_sent(sent, words)])[0]
    # filter out predictions below the error threshold and create tuples of intent and probability for each result
    results = [(classes[i],r) for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by prediction confidence (probability)
    results.sort(key=lambda x: x[1], reverse = True)
    return results

def bot_response(sent, userID = '123', show_details = False):
    '''
    Obtain the response by the model for the given input sentence
    Parameters:
            sent: the input sentence by the user, a scalar string
            userID: the unique ID representing each user, needed to save the context unique to each user
            show_details: a boolean to indicate if the details involved in obtaining the response should be displayed for testing purposes
    Returns: 
            a random choice of response, a scalar string
                the model predicts the given sentence to belong to a specific class, and 
                a response is selected from the array of responses included for that class, based on the context established for the user
    '''
    results = predict_class(sent)
    print(results)
    flag = True
    # find the intent tag for the best result based on CONTEXT
    if results:
        # loop all possible matches to find a context appropriate response
        for match in results:
            for i in intents['intents']:
                # find the tag that is in match
                if i['tag'] == match[0]:
                    if show_details: print('match: ', match)
                    # set the context if it exists for the matched intent
                    if 'context_set' in i:
                        if show_details: print('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # return a response if the matched intent is : 
                    # (1) not contextual, or 
                    # (2) contextual and applies to the given user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        flag = False
                        if show_details: print('tag:', i['tag'])
                        return (random.choice(i['responses']))
    
    # if none of the responses satisfy the requirement, return the deafult response
    if flag:
        i = intents['intents'][0]
        if show_details: print('default_tag:', i['tag'])
        return (random.choice(i['responses']))


# Creating a Graphical User Interface (GUI) using tkinter

In [4]:
# import functions from tkinter
import tkinter
from tkinter import *

def send():
    '''
    take input from the user and send the response obtained from the model
    components used:
        ChatLog: the chat window
        EntryBox: the box inside the chat window to enter the message
        ScrollBar: bind a scrollbar to the chat window
        SendButton: the button for sending the message
    '''
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg !='':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, 'You: '+ msg + '\n\n')
        ChatLog.config(foreground="#442265", font = ('Verdana', 12))
        
        res = bot_response(msg,show_details=True)
        ChatLog.insert(END, 'Bot: ' + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("ChatBot")
base.geometry("375x470")
base.resizable(width=FALSE, height=FALSE)

# create chat window
ChatLog = Text(base, bd=0, bg='white', height='8', width='50', font='Arial')

ChatLog.config(state=DISABLED)

# bind scrollbar to chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor='heart')
ChatLog['yscrollcommand'] = scrollbar.set

# create button to send message
SendButton = Button(base, font=('Verdana', 12, 'bold'), text='Send', width='12', height='5', bd=0, bg='#32de97', activebackground='#3c9d9b', fg='#ffffff', command=send)

# create the box to enter message
EntryBox = Text(base, bd=0, bg='white', width='29', height='5', font='Arial')

#place all components on the screen
scrollbar.place(x=356, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=347)
EntryBox.place(x=128, y=401, height=60, width=265)
SendButton.place(x=6, y=401, height=60)

ChatLog.config(state=NORMAL)
# the first message seen on the chat window
ChatLog.insert(END,"Bot: Hello there! Welcome to STEMforall. I'm a Bot, here to answer any questions you have about our services.\n\n")
ChatLog.config(foreground="#442265", font = ('Verdana', 12))
ChatLog.config(state=DISABLED)

# create an infinite loop used to run the application, 
# wait for an event to occur and process the event as long as the window is not closed.
base.mainloop()


[('greeting', 0.9986443)]
match:  ('greeting', 0.9986443)
tag: greeting
[('STEMforall', 0.98270226)]
match:  ('STEMforall', 0.98270226)
tag: STEMforall
[('partners', 0.99450606)]
match:  ('partners', 0.99450606)
tag: partners
[('lead', 0.9341583)]
match:  ('lead', 0.9341583)
match:  ('lead', 0.9341583)
default_tag: greeting
[('enrollment', 0.9834743)]
match:  ('enrollment', 0.9834743)
tag: enrollment
[('greeting', 0.45839292), ('thanks', 0.33923328)]
match:  ('greeting', 0.45839292)
tag: greeting
[('greeting', 0.45839292), ('thanks', 0.33923328)]
match:  ('greeting', 0.45839292)
tag: greeting
[('tutor_application', 0.98017603)]
match:  ('tutor_application', 0.98017603)
context: ['application']
tag: tutor_application
